안녕하세요! 데이크루 1기로 활동 중인 sssssun입니다!

K-fold와 계층적 K-fold 교차검증법을 다룬 지난 포스팅에 이어, 이번에도 **교차검증(Cross Validaiton)**에 대해 포스팅해볼 예정입니다.

이전에 배웠던 K-fold를 **cross_val_score**와 **cross_validate** 메서드를 통해 구현하는 방법, 그리고 그 외의 **LOOCV**, **반복•그룹별 교차 검증** 등에 대해 공부해보았습니다!

저도 처음 교차검증을 배워보며 공부한 내용을 작성하는 것이기 때문에 다른 팁이나 잘못된 내용이 있다면 꼭 알려주세요😇

### **1. cross_val_score, cross_validate**

sklearn은 model_selection 모듈에서 **cross_val_score** 와 **cross_validate** 메서드를 통해 교차검증을 제공합니다.

기본적으로 cross_val_score 와 cross_validate는 모두 **K-fold 교차검증법**을 따른다고 생각하면 되는데, 모델의 사용에 따라 기본값이 차이가 있습니다.

|사용 모델|교차검증법|
|:------:|:---:|
|회귀|KFold|
|분류|StratifiedKFold|




표에서 나타낸 것과 같이, 모델에 **회귀** 알고리즘을 사용하였을 경우 cross_val_score 와 cross_validate 를 이용하면 **KFold** 교차검증법에 따라 모델을 평가하고, **분류** 알고리즘을 사용하였을 경우 **계층적 KFold** 교차검증법에 따릅니다.

그럼, 지난 포스팅과 마찬가지로 iris dataset의 레이블을 분류하는 모델을 구현해보겠습니다.

우리는 의사결정나무 알고리즘을 이용하여 iris 레이블을 분류하는 모델을 만들었으니, cross_val_score 와 cross_validate 를 이용하면 계층적 k겹 교차검증법에 따를 것입니다.

먼저, 필요한 라이브러리들을 import 해주었습니다.

In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

먼저 cross_val_score를 활용해보겠습니다.

In [8]:
from sklearn.model_selection import cross_val_score

의사결정나무 모델을 객체 지정해주었습니다.

In [18]:
iris = load_iris()
feature = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=1)

cross_val_score 함수 내 매개변수들은 다음과 같습니다.

- estimator : 학습할 모델
- x : 훈련 데이터셋
- y : 레이블 데이터셋
- cv: 폴드세트의 수

매개변수를 함수 내에 넣어주기만 하면 됩니다.

cv 를 설정하지 않으면, 5겹 교차검증이 default 입니다.

그리고 폴드 세트마다 정확도를 array로 저장합니다!

In [20]:
basic_scores = cross_val_score(dt_clf, feature, label)
print('\n##cross_val_score를 통한 성능 평가 : ', basic_scores)


##cross_val_score를 통한 성능 평가 :  [0.96666667 0.96666667 0.9        1.         1.        ]


평균을 내어주었습니다.

In [19]:
print('\n## 평균 검증 정확도:', round(np.mean(basic_scores),4))


## 평균 검증 정확도: 0.9667


정확도가 0.9667이 나오네요.

cross_val_score 가 실제로 계층적 교차검증을 따르는지 확인해보겠습니다.

지난 포스팅에서 다뤘던 StratifiedKFold 메서드를 활용하여 5겹 교차 검증을 통한 최종 모델 성능 정확도를 산출해보았습니다.

In [18]:
from sklearn.model_selection import StratifiedKFold
S_kfold = StratifiedKFold(n_splits=5)

cv_accuracy = []
n_iter =0

for train_index, test_index in S_kfold.split(feature, label):  
    x_train, x_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', round(np.mean(cv_accuracy),4))


## 평균 검증 정확도: 0.9667


위의 cross_val_score 의 결과와 동일하게 최종 정확도가 0.9667가 나오는 것을 확인할 수 있네요!

결과는 같지만, cross_val_score 함수를 사용하는 경우가 코드가 훨씬 간결해보입니다.

다음으로는 **cross_validate**를 사용해볼텐데요.

In [34]:
from sklearn.model_selection import cross_validate

기본적인 매개변수 사용법과 설정값은 cross_val_score 와 같습니다!

다만 차이점은 cross_validate는 분할마다 **훈련과 테스트에 걸린 시간**을 담은 딕셔너리를 반환해줍니다.

또한 **train_score**, 매 세트마다 훈련 결과를 볼 수도 있습니다. 

하지만 train_score는 기본 출력값이 아니기 때문에 보기 위해서는 **return_train_score=True**를 입력해주어야 합니다.

저는 데이터프레임으로 만들어 한눈에 걸린 시간과 정확도를 확인해보았습니다.

In [35]:
c_val = cross_validate(dt_clf, feature, label)

df = pd.DataFrame(c_val, index = ['set1', 'set2', 'set3', 'set4', 'set5'])
df

,fit_time,score_time,test_score
set1,0.003454,0.000591,0.966667
set2,0.001480,0.000570,0.966667
set3,0.001404,0.000506,0.900000
set4,0.001347,0.000511,1.000000
set5,0.001369,0.000465,1.000000


잘 출력이 되는 것을 확인할 수 있습니다.

In [33]:
df.describe().loc[['mean'], :]

,fit_time,score_time,test_score
mean,0.00187,0.000535,0.966667


평균을 내어보았을 때, test_score가 StratifiedKFold, cross_val_score와 모두 동일하게 0.9667이 나오는 것을 볼 수 있습니다.

학습 및 검증 시간, 세트별 정확도를 한눈에 봐야한다면 cross_validate가 좋은 방법이 될 것 같습니다.

### **2. LOOCV(Leave-One-Out Cross-Validation)**

다음으로 소개할 교차검증법은 **LOOCV(Leave-One-Out Cross-Validation)**입니다.

이 방법도 KFold 교차검증법의 일종인데,  LOOCV는 **폴드 하나에 샘플 하나**만 들어있습니다. 

각 반복에서 하나의 데이터 포인트를 선택해 테스트 셋으로 사용합니다. 

즉, 1개의 데이터가 테스트 데이터셋이 되고, 나머지는 모두 훈련 데이터셋이 되는 과정을 데이터개수만큼 반복하는 것이죠.

폴드 하나에 데이터가 하나만 있기 때문에 데이터셋이 클 때는 시간이 오래 걸리지만 작은 데이터셋에서는 좋은 결과를 만들어내는 것이 특징입니다.

먼저 LOO 메서드를 import 해주었습니다.

In [4]:
from sklearn.model_selection import LeaveOneOut

LeaveOneOut 함수의 사용은 모델 알고리즘을 사용할 때처럼 객체 지정을 해주어야 합니다.

그리고 cross_val_score 함수의 **cv** 파라매터에 넣어주면 됩니다.

간단하죠?

In [45]:
iris = load_iris()
feature = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=1)

loocv = LeaveOneOut()
score = cross_val_score(dt_clf, feature, label, cv = loocv)

'교차 검증 횟수'는 **검증을 몇번 반복했는지**를 나타냅니다.

1개의 샘플이 곧 한개의 폴드 세트가 되기 때문에 150개의 데이터로 이루어진 iris dataset은 150번의 검증 과정을 반복한 것입니다. 

In [46]:
print("\n## 교차 검증 횟수 : ", len(score))
print('\n## 평균 검증 정확도:', round(np.mean(score),4))


## 교차 검증 횟수 :  150

## 평균 검증 정확도: 0.94


최종 정확도가 0.94 정도로 나오는 것을 보아, iris datset을 의사결정나무로 분류하는 모델에는 LOOCV 교차검증법이 최선은 아니겠네요.

### **3. 임의 분할 교차 검증(shuffle-split cross-validation)**

다음으로 소개할 교차검증법은 교차 검증은 **shuffle-split cross-validation, 임의 분할 교차 검증**입니다.

임의 분할 교차 검증에서는 **train_size**와 **test_size**를 지정할 수 있는데, 말그대로 각각 훈련 데이터셋과 테스트 데이터셋 사이즈를 의미합니다.

그리고 train_size와 test_size에 **정수**를 입력하면 **데이터의 개수**, **실수**를 입력하면 각각 훈련 데이터셋과 테스트 데이터셋의 **비율**을 의미하죠.


train_size 만큼으로 훈련 세트를 만들고 훈련 세트와 중첩되지 않은 test_size 만큼으로 테스트 세트를 만듭니다.

이렇게 분할된 데이터셋은 지정한 **n_splits 횟수만큼 반복**이 되는 것이죠.

그럼 코드로 구현해볼까요?

In [22]:
from sklearn.model_selection import ShuffleSplit

사용법은 LOOCV와 동일합니다.

ShuffleSplit은 객체 지정해주고, cv 매개변수에 넣어주면 됩니다.

In [41]:
iris = load_iris()
feature = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=1)

shuffle_split = ShuffleSplit(test_size = .6, train_size =.4, n_splits = 20)
score = cross_val_score(dt_clf, feature, label, cv = shuffle_split)

저는 test_size가 60%,  train_size가 40%가 되도록 데이터셋을 분할해주었고, 20번 반복 했습니다. 

shufflesplit은 데이터에서 **무작위로 추출**해서 훈련 데이터셋과 테스트 데이터셋을 만들기 때문에 random_state 매개변수를 지정하지 않으면 실행할 때마다 결과가 다르게 나오는 것 주의하세요!

또한 KFold와 다르게 하나의 데이터가 여러 테스트 데이터셋에 포함될 수 있습니다.

In [43]:
print("\n## 교차 검증 횟수 : ", len(score))
print('\n## 평균 검증 정확도:', round(np.mean(score),4))


## 교차 검증 횟수 :  20

## 평균 검증 정확도: 0.9406


교차 검증 횟수는 지정해준대로 n_splits 의 값과 같은 20이 나왔고, 정확도는 0.9406이 나왔네요.

임의 분할 교차 검증은 **반복 횟수를 훈련 데이터셋이나 테스트 데이터셋의 크기와 독립적으로 조절해야 할 때** 유용하게 사용됩니다. 

데이터셋의 크기와 비율을 어떻게 설정하던지, 반복 횟수는 그 영향을 받지 않으니까요.

또한 train_size와 test_size의 합을 전체 데이터의 개수와 다르게 하거나 비율의 합을 1보다 작게 하여 **전체 데이터의 일부만 사용**할 수도 있습니다.

이런 부분 샘플링은 대규모 데이터 셋으로 작업할 때 도움이 됩니다!

### **4. 반복 교차 검증**

데이터 셋의 크기가 크지 않을 경우 안정된 검증 점수를 얻기 위해 **교차 검증을 반복**해서 여러 번 수행할 수 있습니다. 

sklearn에서 교차검증을 반복해서 수행해주는 메서드가 바로 **RepeatedKFold**와 **RepeatedStratifiedKFold** 입니다.


이름에서도 알 수 있듯이, RepeatedKFold와 RepeatedStratifiedKFold 둘다 KFold의 일종인데 이때 **회귀** 모델에는 **RepeatedKFold**를 사용하고 **분류** 모델에는 **RepeatedStratifiedKFold**를 사용해야 합니다.

우리는 계속해서 의사결정나무 알고리즘을 이용하여 iris 레이블을 분류하고 있기 때문에 RepeatedStratifiedKFold를 사용하겠습니다.

In [24]:
from sklearn.model_selection import RepeatedStratifiedKFold

마찬가지로, 메서드의 객체를 cross_val_score 함수의 **cv** 매개변수에 입력하면 됩니다!
​

fold set를 **몇개로 분할**할 것인지는 **n_split** 매개변수를 사용하고, **반복 횟수**는 **n_repeat** 매개변수를 사용합니다.

n_split의 default 값은 5, n_repeat의 default 값은 10입니다.

In [33]:
iris = load_iris()
feature = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=1)

rskfold = RepeatedStratifiedKFold()
score = cross_val_score(dt_clf, feature, label, cv = rskfold)

검증 횟수는 **분할 개수X반복 횟수(n_spllitXn_repeat)**가 됩니다.

In [34]:
print("\n## 교차 검증 횟수 : ", len(score))
print('\n## 평균 검증 정확도:', round(np.mean(score),4))


## 교차 검증 횟수 :  50

## 평균 검증 정확도: 0.946


교차 검증 횟수는 5X10 = 50이고, 정확도는 0.946 정도이네요.

### **5. 그룹 교차 검증(groups cross-validation)**

추가적으로, **데이터 안에 연관된 데이터가 그룹으로 있을 경우** 사용하는 **그룹 교차 검증(groups cross-validation)**을 소개하겠습니다.

**GroupKFold** 메서드를 사용할 수 있고, groups array를 만들어서 훈련 데이터셋과 테스트 데이터셋을 만들 때 같은 그룹으로 지정한 데이터들이 분리되지 않고 **동시에 훈련 데이터셋이나 테스트 데이터셋에 들어갈 수 있도록** 합니다.

소개했던 다른 교차검증법들과 마찬가지로, kFold를 기반으로 합니다.

In [35]:
from sklearn.model_selection import GroupKFold

iris dataset에는 사용하기 적절하지 않기 때문에 iris 분류 모델을 통한 실습을 생략하였습니다.

이렇게, 추가적으로 sklearn을 통해 사용할 수 있는 5가지 교차 검증법에 대해 더 알아보았습니다.

의사 결정 나무 알고리즘을 이용한 iris dataset 분류 모델에서는 계층적 교차검증법이 정확도가 제일 높았습니다.

사용할 수 있는 교차 검증법이 정말 다양하니, 데이터셋이나 사용하는 알고리즘에 따라서 가장 최적의 교차 검증법을 사용하면 될 것 같아요!

cross_val_score 를 통해 cv 부분만 수정해주면 되는 것이 간편하고 이해하기 쉬웠습니다.

읽어주셔서 감사합니다!:)